In [ ]:
# # no need for this since it has nothing outside classes and functions
# print(__name__)
# if __name__ == "__main__" and hasattr(__builtins__,'__IPYTHON__') and ('google.colab' in str(get_ipython())):
#     from google.colab import drive
#     drive.mount('/content/drive')
#     %cd /content/drive/MyDrive/PressureReliefWorkArea/SummerWork/
#     !ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/PressureReliefWorkArea/SummerWork
AbstractModel.ipynb  DataProcessing.ipynb   RunAndEvaluation.ipynb
CNNModel.ipynb	     HelperFunctions.ipynb


In [ ]:
# from abc import ABC, abstractmethod


from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


# -n does not work apparently??
    # __name__ == "__main__" even with -n.
# I commented under an issue logged on IPython's GitHub repository requesting this fix,
    # and I will be reposting the issue if it is not seen
    # (since that issue is very old and appears to have been left unseen so far)
%run -n DataProcessing.ipynb
# but apparently the ipynb module hates running %cd and !ls AND %run in an imported notebook???
    # I can't even import a second ipynb file from inside the first import...
# !pip install ipynb
# import ipynb
# from ipynb.fs.full.DataProcessing import *

In [ ]:
class SKJFModel:
    def __init__(self, sequence_length = None):
        super(SKJFModel, self).__init__()
        # self.loader = JFLoader(file_path, sequence_length)
        self.sequence_length = sequence_length
        self.train_class_type = None
        self.train_loader = None
        self.__eval_class_type = None
        self.__eval_loader = None
        self.__counter = 0
        self.__label_converter = None

    def get_eval_loader(self):
        if self.__eval_loader is None:
            return self.train_loader
        return self.__eval_loader

    def get_eval_class_type(self):
        if self.__eval_class_type is None:
            return self.train_class_type
        return self.__eval_class_type
    
    def get_outputs(self, inputs, feedback = False):
        if self.__counter >= 5:
            raise RecursionError("SKJFModel.get_outputs() has detected what may be an infinite loop")
        self.__counter += 1
        outputs = type(self).get_outputs(self, inputs)
        self.__counter -= 1
        # if self.train_class_type == 3 and self.__eval_class_type == 5:
        #     # print(np.shape(outputs))
        #     outputs = self.__label_converter.convert_label_type(outputs, False, False, True, torch.Tensor, feedback)
        #     # print(np.shape(outputs))
        return outputs


        # raise NotImplementedError(f"SKJFModel.__get_outputs() is an abstract method and should be defined before used.")

    def change_io_specs(self, class_type = None, *args, **kwargs):
        if(class_type != 5 or self.train_class_type != 3):
            raise NotImplementedError("Yeah, this function was cobbled together. I strongly suggest putting other loading logic into place in places besides here before trying to work on it")
        # validate_conversion(self.train_class_type, class_type)
        if args:
            repress_classes = args[0]
            if len(args) > 1:
                args = args[1:]
            else:
                args = []
        else:
            repress_classes = kwargs.pop("repress_classes", True)

        self.__eval_class_type = class_type
        self.__eval_loader = JFSKLoader(SKFileNameHandler.get_filename_for_classtype(self.train_class_type + 2), self.sequence_length, repress_classes, *args, **kwargs)
        self.__label_converter = SKLabelConverter(None, self.train_loader.specifier_values)
        self.__label_converter.set_label_type_conversion(5 - 2, 3 + 2)



    def load_model(self, file_path, class_type):
        raise NotImplementedError(f"SKJFModel.load_model() is an abstract method and should be defined before used.")


    def train(self):
        raise NotImplementedError(f"SKJFModel.train() is an abstract method and should be defined before used.")


    def load_data(self, file_path, *args, **kwargs):
        if args:
            repress_classes = args[0]
            if len(args) > 1:
                args = args[1:]
            else:
                args = []
        else:
            repress_classes = kwargs.pop("repress_classes", True)
        self.train_loader = JFSKLoader(file_path, self.sequence_length, repress_classes, *args, **kwargs)
        class_type = self.train_loader.specifier_values[SKDescriptors.CLASSIFICATION_TYPE_FS]
        # assertion
        SKDescriptors.validate_class_type(class_type)
        # needed definitions
        self.train_class_type = class_type


    def evaluate(self, show_cm = True):
        all_preds = []
        all_labels = []
        correct = 0
        total = 0
        first_iteration = True

        counter = 0
        with torch.no_grad():   # Since we're not training, we don't need to calculate the gradients
            for data in self.get_eval_loader().test_dataloader:
                inputs, labels = data
                outputs = SKJFModel.get_outputs(self, inputs, first_iteration)
                if first_iteration:
                    first_iteration = False
                _, predicted = torch.max(outputs.data, 1)
                if self.train_class_type != self.get_eval_class_type():
                    predicted = self.__label_converter.convert_label_type(predicted, False, False, True, torch.Tensor, first_iteration)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                if not counter % 10:
                    print("click")
                    print(predicted.cpu().numpy())
                    print(labels.cpu().numpy())
                counter += 1
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the test data: %d %%' % (100 * correct / total))
        
        # self.jfm_plot_confusion_matrix(show_cm)
        if show_cm:
            class_names = SKDescriptors.CTS_PER_TYPE[self.get_eval_class_type()] # [str(c) for c in SKDescriptors.CTS_PER_TYPE[self.class_type]]
            cm = confusion_matrix(all_labels, all_preds)

            fig, ax = plt.subplots(figsize=(10, 10))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
            plt.ylabel('Actual')
            plt.xlabel('Predicted')
            plt.title('Confusion Matrix')
            plt.show()